<a href="https://colab.research.google.com/github/Santiago8888/Kin/blob/master/Job_s_PCA_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
cd gdrive/My Drive/Dzeta

/content/gdrive/My Drive/Dzeta


In [0]:
import json
with open('Data/angel_V1.json') as json_file:
    data = json.load(json_file)

In [0]:
import pandas as pd
df = pd.DataFrame.from_records(data[1:], columns=data[0])

In [0]:
descriptions = df['description'].tolist()
descriptions = sum(descriptions, [])

In [7]:
import nltk
import torch
import pandas as pd

import numpy as np
from sklearn.cluster import KMeans

#!pip install nmslib 
import nmslib

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
cd NLP

/content/gdrive/My Drive/Dzeta/NLP


In [10]:
from InferSent.models import InferSent
model_version = 2
MODEL_PATH = "encoder/infersent%s.pkl" % model_version
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048, 'pool_type': 'max', 'dpout_model': 0.0, 'version': model_version}

model = InferSent(params_model)
model.load_state_dict(torch.load(MODEL_PATH))

W2V_PATH = 'fastText/crawl-300d-2M.vec'
model.set_w2v_path(W2V_PATH)
model.build_vocab(descriptions, tokenize=True)
embeddings = model.encode(sentences=descriptions, tokenize=True)


Found 3245(/3486) words with w2v vectors
Vocab size : 3245


In [91]:
df.describe()

,id,date,minSalary,maxSalary,minEquity,maxEquity,salary,equity,locationsIdx,applicantsIdx,jobsIdx,isEuro,isPound,latitude,longitude
count,8.300000e+01,8.300000e+01,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,76.000000,76.000000
mean,4.066884e+06,1.573491e+09,0.237070,0.253202,0.044400,0.078217,0.245729,0.066945,1.277108,31.481928,2.469880,0.072289,0.144578,0.637712,0.192234
std,2.476070e+06,1.570184e+07,0.167929,0.196964,0.131052,0.142226,0.177761,0.134606,1.192591,63.650218,1.921238,0.260540,0.353813,0.244441,0.230007
min,2.777300e+04,1.508388e+09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.573026e+06,1.560841e+09,0.131098,0.052632,0.000000,0.000000,0.081921,0.000000,1.000000,2.000000,1.000000,0.000000,0.000000,0.498857,0.002925
50%,4.399507e+06,1.582071e+09,0.207317,0.263158,0.001000,0.015000,0.265537,0.013333,1.000000,10.000000,2.000000,0.000000,0.000000,0.593278,0.106923
75%,6.153710e+06,1.585310e+09,0.283537,0.421053,0.020000,0.100000,0.343220,0.073333,1.500000,32.000000,3.000000,0.000000,0.000000,0.924373,0.287661
max,7.591107e+06,1.585902e+09,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,7.000000,365.000000,10.000000,1.000000,1.000000,1.000000,1.000000


In [0]:
df["minSalary"] = ((df["minSalary"]-df["minSalary"].min())/(df["minSalary"].max()-df["minSalary"].min()))
df["maxSalary"] = ((df["maxSalary"]-df["maxSalary"].min())/(df["maxSalary"].max()-df["maxSalary"].min()))
df["minEquity"] = ((df["minEquity"]-df["minEquity"].min())/(df["minEquity"].max()-df["minEquity"].min()))
df["maxEquity"] = ((df["maxEquity"]-df["maxEquity"].min())/(df["maxEquity"].max()-df["maxEquity"].min()))
df["salary"] = ((df["salary"]-df["salary"].min())/(df["salary"].max()-df["salary"].min()))
df["equity"] = ((df["equity"]-df["equity"].min())/(df["equity"].max()-df["equity"].min()))


In [0]:
df['isPound'] = df['compensation'].str.contains('£')*1
df['isEuro'] = df['compensation'].str.contains('€')*1


In [58]:
df.head()

,provider,id,description,date,datePosted,location,title,link,minSalary,maxSalary,minEquity,maxEquity,salary,equity,compensation,name,logo,pitch,size,locationsIdx,applicantsIdx,jobsIdx,isEuro,isPound
0,Angel.co,6243347,[Ubico provides an AI-powered lead engagement ...,1581106732,2020-02-07T20:18:52.000Z,Montreal,Full-Stack Engineer,https://angel.co/company/ubico-mtl/jobs,0.085366,0.026316,0.10,0.150,0.053672,0.133333,$50k – $65k • 0.5% – 1.5%,Ubico,https://photos.angel.co/startups/i/6243347-703...,AI-Powered Lead Engagement Platform,SIZE_11_50,1,3,4,0,0
1,Angel.co,2742580,[Only applications submitted via https://jobs....,1555718139,2019-04-19T23:55:39.000Z,San Francisco,Software Engineer,https://angel.co/company/roger-ai-1/jobs,0.207317,0.421053,0.00,0.010,0.322034,0.006667,$70k – $140k • 0.0% – 0.1%,Roger,https://photos.angel.co/startups/i/2742580-01a...,Accounting automation tool for SMBs and Accoun...,SIZE_11_50,0,26,0,0,0
2,Angel.co,5716138,"[**About Odyssean**\r, With over 50 years of c...",1585377380,2020-03-28T06:36:20.000Z,Remote,Backend Engineer,https://angel.co/company/odyssean/jobs,0.146341,0.263158,0.40,0.500,0.209040,0.466667,$60k – $110k • 2.0% – 5.0%,Odyssean,https://photos.angel.co/startups/i/5716138-fe3...,"Market making, trading, and digital asset mana...",SIZE_1_10,7,167,3,0,0
3,Angel.co,5716138,"[**About Odyssean**\r, Odyssean is a digital a...",1585376095,2020-03-28T06:14:55.000Z,New York City,CTO,https://angel.co/company/odyssean/jobs,0.573171,0.684211,1.00,1.000,0.632768,1.000000,$130k – $190k • 5.0% – 10.0%,Odyssean,https://photos.angel.co/startups/i/5716138-fe3...,"Market making, trading, and digital asset mana...",SIZE_1_10,7,167,3,0,0
4,Angel.co,233847,[We are seeking motivated developers who are p...,1551993244,2019-03-07T21:14:04.000Z,Houston,Software Engineer,https://angel.co/company/harvest-exchange/jobs,0.390244,0.315789,0.02,0.025,0.350282,0.023333,$100k – $120k • 0.1% – 0.25%,Harvest Exchange,https://photos.angel.co/startups/i/233847-bc53...,Helping Investors Grow,SIZE_11_50,2,2,2,0,0


In [0]:
df_locations = df.location.unique()

In [0]:
with open('Data/locations.json') as json_file:
    locations = json.load(json_file)

In [67]:
for d in df_locations: print(locations[d])

[45.4972159, -73.6103642]
[37.7790262, -122.4199061]
[None, None]
[40.7127281, -74.0060152]
[29.7589382, -95.3676974]
[42.4979693, -92.3329637]
[34.0536909, -118.2427666]
[51.5073219, -0.1276474]
[43.6534817, -79.3839347]
[51, 10]
[39.7392364, -104.9848623]
[22.2793278, 114.1628131]
[34.0211224, -118.3964665]
[30.2711286, -97.7436995]
[25.7742658, -80.1936589]
[37.4429964, -122.1545229]
[37.7274692, -89.216655]
[53.3497645, -6.2602732]
[51.0534234, -114.0625892]
[37.2333253, -121.6846349]
[52.5170365, 13.3888599]
[32.7174209, -117.1627714]
[50.8220399, -0.1374061]
[42.3602534, -71.0582912]
[40.748736, -73.98486]
[49.2608724, -123.1139529]
[48.8566969, 2.3514616]
[51.0000002, -109.0000002]
[47.6038321, -122.3300624]


In [0]:
df['latitude'] = [locations[d][0] for d in df['location']]
df['longitude'] = [locations[d][1] for d in df['location']]

In [0]:
df["latitude"] = ((df["latitude"]-df["latitude"].min())/(df["latitude"].max()-df["latitude"].min()))
df["longitude"] = ((df["longitude"]-df["longitude"].min())/(df["longitude"].max()-df["longitude"].min()))